In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
!sm-docker build . --repository ngc:byoc

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ngc'
tag = ':byoc'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'

byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
# byoc_image_uri = '138604873012.dkr.ecr.ap-southeast-1.amazonaws.com/ngc:byoc'

instance_type='ml.g4dn.xlarge'

estimator = Estimator(image_uri=byoc_image_uri,
                      role=get_execution_role(),
                      base_job_name='tf-custom-container-test-job',
                      instance_count=1,
                      instance_type=instance_type)

# start training
estimator.fit()


In [ ]:
estimator.model_data

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data=estimator.model_data, 
                        role=get_execution_role(),
                       framework_version='2.4')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
# # deploy the trained model
# predictor = estimator.deploy(1, 'm5.large')

In [ ]:
# Predict
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

# cell 12
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))
    

In [ ]:
# Delete endpoint
predictor.delete_endpoint()